In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import numpy as np
from pyuvdata import UVCal, UVData
import os
import sys
import glob
import uvtools as uvt
from astropy.time import Time
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, Angle
import pandas
import warnings 
import copy
from hera_notebook_templates import utils
import hera_qm
from hera_mc import cm_hookup
import h5py
#warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'hera_notebook_templates'

In [2]:
# get data location
JD = os.environ['JULIANDATE']
data_path = os.environ['DATA_PATH']
ant_metrics_ext = os.environ['ANT_METRICS_EXT']
print(f'JD = {JD}')
print(f'data_path = "{data_path}"')
print(f'ant_metrics_ext = "{ant_metrics_ext}"')

JD = 2458938
data_path = "/lustre/aoc/projects/hera/H3C/IDR3_1/2458938"
ant_metrics_ext = ".all_ants.ant_metrics.hdf5"


In [3]:
# Load in data
HHfiles, difffiles, uvdx, uvdy = utils.load_data(data_path,JD)
uvd = UVData()
uvd.read(HHfiles, ant_str = 'autos', skip_bad_files=True)
lsts = uvd.lst_array

NameError: name 'utils' is not defined

## LST Coverage

Shows the LSTs (in hours) and JDs for which data is collected. Green represents data, red means no data.

In [4]:
utils.plot_lst_coverage(uvd)

NameError: name 'utils' is not defined

## Autocorrelations for a single file

This plot shows autocorrelations for one timestamp of each antenna that is active and each polarization

In [5]:
### plot autos
utils.plot_autos(uvdx, uvdy)

NameError: name 'utils' is not defined

## Waterfalls of Autocorrelation Amplitudes for each Antenna and Each polarization

In [6]:
utils.plot_wfs(uvd, pol = 0)

NameError: name 'utils' is not defined

In [7]:
utils.plot_wfs(uvd, pol = 1)

NameError: name 'utils' is not defined

## Correlation Metrics

The first plot shows the correlation metric (described below) for a set of baseline types, as calculated at several times throughout the night. It is expected that longer baselines (darker color) will exhibit lower values than the short baselines. 

The matrices show the phase correlation between antennas. Using the even and odd visibilities, each pixel is calculated as (even/abs(even)) * (conj(odd)/abs(odd)), and then averaged across time and frequency. If the phases are noise-like, this value will average down to zero. If the antennas are well correlated, the phases should not be noise-like, and this value should average to 1. The lines denoting node boundaries are intended to help confirm that inter-node correlations are functioning - if they aren't, this plot will appear block-diagonal.

This metric has shown to be LST locked - when comparing to other nights, be sure to compare for the same LST. It is expected that some LSTs will look much better or worse than others.

Note: Within each node, the order of antennas is determined by snap, and within that by snap input number. 

In [8]:
badAnts = []
badAnts = utils.plotNodeAveragedSummary(uvd,HHfiles,JD)

NameError: name 'utils' is not defined

Visibility amplitude spectra for a set of redundant baselines, labeled by inter vs. intranode baselines. The red and blue should exhibit the same bandpass shape - if the red are consistently different from the blue, this indicates an issue with internode correlations.

Note: antennas that were identified as bad by the correlation matrix have been removed from this plot.

In [9]:
utils.plotVisibilitySpectra(HHfiles[len(HHfiles)//2], JD, badAnts=badAnts)

NameError: name 'utils' is not defined

## Antenna Positions

Antennas outlined in black here have been identified by the correlation matrix as bad antennas. 

In [10]:
uvd1 = UVData()
uvd1.read(HHfiles[5], skip_bad_files=True)
utils.plot_antenna_positions(uvd1, badAnts=badAnts)

NameError: name 'HHfiles' is not defined

## Even and Odd File Checks

A waterfall showing the ratio between the even and odd visibilities. The purpose of this is to highlight xengine failures, which will cause this value to fall to zero or go to infinity. If things are working properly, this value should be stable at 1. The boundaries between different x-engines are shown by the vertical red lines.

In [11]:
if len(HHfiles) == len(difffiles):
    uvd_diff = UVData()
    uvd_diff.read(difffiles, ant_str = 'autos', skip_bad_files=True)
    utils.plotEvenOddWaterfalls(uvd,uvd_diff)
else:
    uvd_diff = UVData()
    use_diffs = [f for f in difffiles if '%s/zen.%s.%s.sum.uvh5' % (data_path,f.split('.')[1],f.split('.')[2]) in HHfiles]
    uvd_diff.read(use_diffs, ant_str = 'autos', skip_bad_files=True)
    uvd_sum = uvd.select(times=np.unique(uvd_diff.time_array),inplace=False)
    utils.plotEvenOddWaterfalls(uvd_sum,uvd_diff)

NameError: name 'HHfiles' is not defined

## Antenna Metrics Over a Whole Night

In [12]:
ant_metrics_list = [df.replace('.uvh5', ant_metrics_ext) for df in HHfiles]
apriori_exants = {}
am_xants_dict = {}
am_dead_ants_dict = {}
am_crossed_ants_dict = {}
for amf in ant_metrics_list:
    with h5py.File(amf, "r") as infile:
        xants = infile['Metrics']['xants'][:]
        dead_ants = infile['Metrics']['dead_ants'][:]
        crossed_ants = infile['Metrics']['crossed_ants'][:]        
        ex_ants_string = infile['Header']['history'][()].decode()
        ex_ants_string = ex_ants_string.split('--apriori_xants')[1]
        ex_ants_string = ex_ants_string.split('--')[0].strip()
        
    am_xants_dict[amf] = [(int(ant[0]), ant[1].decode()) for ant in xants]
    am_dead_ants_dict[amf] = [(int(ant[0]), ant[1].decode()) for ant in dead_ants]
    am_crossed_ants_dict[amf] = [(int(ant[0]), ant[1].decode()) for ant in crossed_ants]
    apriori_exants[amf] = [int(ant) for ant in ex_ants_string.split()]


file = HHfiles[0]
uvd_hh = UVData()
uvd_hh.read_uvh5(file)
uvdx = uvd_hh.select(polarizations = -5, inplace = False)
uvdx.ants = np.unique(np.concatenate([uvdx.ant_1_array, uvdx.ant_2_array]))
antstemp = uvdx.get_ants()
ants = []

for ant in antstemp:
    ants.append((ant,'Jee'))
    ants.append((ant, 'Jnn'))

NameError: name 'HHfiles' is not defined

In [13]:
exants_grid = np.zeros((len(ants),len(HHfiles))) + 4.0
ant_flags_dict = []

# 0 = apriori, 1 = AM dead, 2 =  AM crossed, 3 = redcal, 4 = no flags 
for f, cal in enumerate(ant_flags_dict):
    for i, ant in enumerate(ants):
        if ant_flags_dict[cal][ant]:
            exants_grid[i, f] = 3.0
for f, amf in enumerate(am_xants_dict):
    for i, ant in enumerate(ants):
        if ant in am_dead_ants_dict[amf]:
            exants_grid[i, f] = 1.0                
        elif ant in am_crossed_ants_dict[amf]:
            exants_grid[i, f] = 2.0
for f, amf in enumerate(apriori_exants):
    for i, ant in enumerate(ants):
        if ant[0] in apriori_exants[amf]:
            exants_grid[i, f] = 0.0

from mpl_toolkits.axes_grid1 import make_axes_locatable
color_order = ['gold', 'magenta', 'cyan', 'green']
cmap = matplotlib.colors.ListedColormap(color_order)
bounds=np.arange(len(color_order)+1)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
            
fig = plt.figure(figsize=(14,len(ants)/4.625))
im = plt.imshow(exants_grid, aspect='auto', cmap=cmap, norm=norm,
                extent=[lsts[0]*12/np.pi, lsts[-1]*12/np.pi, len(ants), 0])
plt.yticks(np.arange(len(ants))+.5, 
           labels = ['{}{}'.format(ant[0], ant[1][-1]) for ant in ants]);
plt.grid(color='w', linestyle='-', linewidth=1.5, axis='y')
plt.xlabel('LST (Hours)')

divider = make_axes_locatable(plt.gca())
cax = divider.new_vertical(size=.4, pad=0.5, pack_start=False)
fig.add_axes(cax)
cbar = fig.colorbar(im, cax=cax, orientation="horizontal")
cbar.set_ticks(.5 + np.arange(len(color_order)))
cbar.set_ticklabels(['Flagged A Priori', 
                     'Flagged by AM as Dead',
                     'Flagged by AM as Crossed',
                     'No Flags', ])

NameError: name 'ants' is not defined

## Delay spectrum

Delay spectrum CLEANed using uvtools.dspec.high_pass_fourier_filter with 7th-order Blackman-Harris window function and full frequency band. All delay spectra except for waterfalls are incoherently averaged over ~1h (if the observation is longer than 1h) with odd/even visibilities to remove noise bias.

In [14]:
# bls = [(ant,ant) for ant in np.sort(uvd.get_ants())]
# _data_sq, data_rs, uvd_ds, uvd_diff = utils.clean_ds(HHfiles, difffiles, bls)

Diagnosis of delay spectra for three different delay regions:

Top panel: compute delay spectrum over 250-500 ns

Middle panel: compute standard deviation ratio $\sigma_{2500-3000}/\sigma_{3000-3200}$ to see the presence of spike at 2500-3000 ns. ~1 in y-axis (dashed line) means there is no spike, and >1 may indicate the presence of spike in the region

Bottom panel: compute distance between delay spectrum and noise level at large delay (3500-4000 ns). ~0 (dashed line) means the delay spectrum can go down to the noise level

In [15]:
# utils.plot_ds_diagnosis(uvd_diff, _data_sq)

In [16]:
# utils.plot_ds_nodes(uvd_diff, _data_sq)

In [17]:
# utils.plot_wfds(uvd_ds, _data_sq, 'ee')

In [18]:
# utils.plot_wfds(uvd_ds, _data_sq, 'nn')

These plots show incoherently averaged (~1h) delay spectra, autocorrelations and clean residuals for each antenna.

Left panel: delay spectra of autocorrelations, averaged noises from diff files (|$<\tilde{\Delta}>_{\rm t}$|, thiner fluctuating lines). The variance of the delay spectrum is also shown ($\sqrt{<|\tilde{\Delta}|^2>_{\rm t}/(2N_{\rm t})}$) which is consistent with the average of diff. Three shaded zones are those explained in the above diagnosis plots.
    
Top right: autocorrelations w/o and w/ flagging. Autocorrelations are normalized to have median=1. Flagged ones are shifted for clarity.
    
Bottom right: clean residuals normalized by the autocorrelations. If there are unflagged RFIs, which may affect the delay spectrum, we may see remaining RFI more clearly in the plot.

In [19]:
# utils.plot_ds(uvd_ds, uvd_diff, _data_sq, data_rs)

In [20]:
# bls_cr = [(uvd.get_ants()[0],ant) for ant in np.sort(uvd.get_ants())][1:]
# _data_sq_cr, data_rs_cr, uvd_cr, uvd_diff_cr = utils.clean_ds(HHfiles, difffiles, bls_cr)

Waterfalls for delay spectrum of crosscorrelation with a reference antenna

In [21]:
# utils.plot_wfds_cr(uvd_cr, _data_sq_cr, 'ee')

In [22]:
# utils.plot_wfds_cr(uvd_cr, _data_sq_cr, 'nn')